## **3. Pondération statistique** (TF-IDF / OKapiBM25)  

https://stackoverflow.com/questions/46580932/calculate-tf-idf-using-sklearn-for-n-grams-in-python  
http://scikit-learn.sourceforge.net/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn-feature-extraction-text-tfidfvectorizer  
https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

https://pypi.org/project/rank-bm25/

In [104]:
path = '../04-filtrage/output/'
acteur = 'inesss'
tag = ''

if tag:
    csv_file = acteur + '_' + tag + '_significant-collocations.csv'

else:
    csv_file = acteur + '_significant-collocations.csv'

### **Lire le vocabulaire** (termes retenus au prétraitement)

In [105]:
from pandas import *

with open(path+csv_file, encoding='utf-8') as f:
    csv = read_csv(f).drop(columns = ['Unnamed: 0'])
    # On va traiter les apostrophes et les parenthèses avec le underscore ("_") ; le tokenizer de sklearn ne les aime pas 
    csv['Terme formatté'] = csv["Terme"].apply(lambda x: str(x).replace("'", "_").replace("(", "_").replace(")", "_").replace("-", "_"))

csv

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm,Terme formatté
0,traitement,NOM,7872,-,-,1218,False,True,traitement
1,jamp,NOM,7296,-,-,992,False,False,jamp
2,évaluation,NOM,6785,-,-,1396,False,False,évaluation
3,médicaments,NOM,5448,-,-,1206,False,True,médicaments
4,demande,NOM,3662,-,-,280,False,False,demande
...,...,...,...,...,...,...,...,...,...
6020,degré,NOM,16,-,-,14,False,False,degré
6021,foam,NOM,16,-,-,8,False,False,foam
6022,coque,NOM,16,-,-,22,False,True,coque
6023,médicaments du québec,NOM PRP:det NOM,16,88.74281960037526,4.496265284875781e-21,13,False,False,médicaments du québec


In [106]:
vocabulaire = [t.lower() for t in list(csv['Terme formatté'])]

In [107]:
print('On a un vocabulaire de {} formes.'.format(len(vocabulaire)))

On a un vocabulaire de 6025 formes.


In [108]:
vocabulaire

['traitement',
 'jamp',
 'évaluation',
 'médicaments',
 'demande',
 'inesss',
 'attente',
 'injection',
 'ministre',
 'patients',
 'médicament',
 'cours',
 'inscription',
 'sodium',
 'citrate',
 'attente d_évaluation',
 'cancer',
 'angita',
 'darunavir',
 'calcium',
 'lurasidone',
 'bicarbonate',
 'réponses',
 'dextrose',
 'sulfonate',
 'bicarbonate de sodium',
 'glycopyrrolate',
 'cours d_évaluation',
 'réponses rapides',
 'covid_19',
 'santé',
 'indication',
 'réponses rapides disponibles',
 'pré_ac',
 'services',
 'listes',
 'décision',
 'québec',
 'nom',
 'demande en attente',
 'coll',
 'recommandations',
 'adultes',
 'projet',
 'fabricant',
 'cliniques',
 'chlorhydrate',
 'soins',
 'forme',
 'dépôt',
 'pré_ac en attente',
 'conseil',
 'recommandation',
 'fumarate',
 'demande en cours',
 'demande en cours d_évaluation',
 'commune',
 'dénomination commune',
 'exception',
 'dénomination',
 'évaluations',
 'maladie',
 'listes des médicaments',
 'canada',
 'usage',
 'ans',
 'nom du fab

### **Lire le corpus**

In [109]:
import os, shutil, re
from pathlib import Path
from os import path
from pandas import *

base_path = '../03-corpus/2-data/1-fr/'
if tag:
    base_path = path.join(base_path, acteur, acteur + '_' + tag + '.csv')

else:
    base_path = path.join(base_path, acteur +  '.csv')
        
with open(base_path, "r", encoding = "UTF-8") as f:
    data = read_csv(base_path, sep=',')
    text = data['text'].tolist()

In [110]:
text = text[:round(len(text))]

nb_docs = len(text)

print("On a donc un corpus de {} documents.".format(nb_docs))

On a donc un corpus de 2035 documents.


### **Nettoyage**

In [111]:
corpus = [str(t).strip('\n').lower().replace('’', '\'') for t in text]
    
punct = '[!#$%&•►*+,;<=>?@[\]^_{|}~©«»—“”–—]'
spaces = '\s+'
postals = '([a-zA-Z]+\d+|\d+[a-zA-Z]+)+'
phones = '\d{3}\s\d{3}-\d{4}' #très simple (trop)

corpus = [str(t).strip('\n').lower().replace('’', '\'') for t in corpus]
corpus = [re.sub(spaces, ' ', t) for t in corpus]
corpus = [re.sub(punct, ' ', t).replace("' ", "'" ) for t in corpus]
corpus = [re.sub(phones, ' STOP ', t) for t in corpus]
corpus = [re.sub(postals, ' STOP ', t) for t in corpus]
corpus = [t.replace("  ", " " ) for t in corpus]
corpus = [t.replace("'", "_").replace("(", "_").replace(")", "_").replace("-", "_") for t in corpus]


On va commencer par utiliser le CountVectorizer pour valider que l'implémentation de sklearn arrive bien au même compte que nous 

In [112]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(vocabulary=vocabulaire, ngram_range=(1,10), token_pattern='\w+')
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()

features_names = vectorizer.get_feature_names_out()
corpus_index = [corpus.index(n) for n in corpus]

import pandas as pd
df = pd.DataFrame(X.T.todense(), index=features_names, columns=corpus_index).transpose()

In [113]:
csv['TF (sklearn)'] = 0
for t in vocabulaire:
    csv.loc[csv['Terme formatté'] == t, 'TF (sklearn)'] = df[t].sum()


csv['DF (sklearn)'] = 0
for t in vocabulaire:
    freqdoc = len(df[df[t] != 0])
    csv.loc[csv['Terme formatté'] == t, 'DF (sklearn)'] = freqdoc
csv

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm,Terme formatté,TF (sklearn),DF (sklearn)
0,traitement,NOM,7872,-,-,1218,False,True,traitement,7864,1180
1,jamp,NOM,7296,-,-,992,False,False,jamp,2556,992
2,évaluation,NOM,6785,-,-,1396,False,False,évaluation,1694,324
3,médicaments,NOM,5448,-,-,1206,False,True,médicaments,5451,1206
4,demande,NOM,3662,-,-,280,False,False,demande,3662,240
...,...,...,...,...,...,...,...,...,...,...,...
6020,degré,NOM,16,-,-,14,False,False,degré,16,14
6021,foam,NOM,16,-,-,8,False,False,foam,16,8
6022,coque,NOM,16,-,-,22,False,True,coque,16,12
6023,médicaments du québec,NOM PRP:det NOM,16,88.74281960037526,4.496265284875781e-21,13,False,False,médicaments du québec,16,13


In [114]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import RegexpTokenizer

#toknizer = RegexpTokenizer()

# max_df : ignore words that appear in 85% of documents, 
# min df:  ignore words that appear in less than 1% of documents 
# vocabulary = vocabulaire

# Sans utiliser le vocabulaire
# tfidf = TfidfVectorizer(min_df=0.1, stop_words=None, ngram_range=(2,4), max_df=0.85, use_idf=True)

# vocabulary = vocabulaire
tfidf_vectorizer = TfidfVectorizer(vocabulary=vocabulaire, ngram_range=(1,10), lowercase=False, token_pattern='\w+')
tfidf = tfidf_vectorizer.fit_transform(corpus)

In [115]:
features_names = tfidf_vectorizer.get_feature_names_out()
corpus_index = [corpus.index(n) for n in corpus]

import pandas as pd
df = pd.DataFrame(tfidf.T.todense(), index=features_names, columns=corpus_index).transpose()

In [116]:
terms_tfidf = {term: df[term].max() for term in df}
zeros = {term : value for term, value in terms_tfidf.items() if terms_tfidf[term] == 0}

In [117]:
zeros

{'bromhydrate': 0.0,
 'sulfonate calcique': 0.0,
 'auteur': 0.0,
 'projet auteur': 0.0,
 'évaluation keytruda': 0.0,
 'hospitalisation et projections des besoins': 0.0,
 'hospitalisation et projections': 0.0,
 'hospitalisation et projections des besoins hospitaliers': 0.0,
 'évaluation keytruda pembrolizumab merck': 0.0,
 'évaluation keytruda pembrolizumab': 0.0,
 'entretien': 0.0,
 'inspq visitez': 0.0,
 'intérêt visitez': 0.0,
 'covid_19 longue': 0.0,
 'évolution de l_épidémie': 0.0,
 'épidémie de covid_19': 0.0,
 'évolution de l_épidémie de covid_19': 0.0,
 'alcool': 0.0,
 'exception décision': 0.0,
 'exception décision du ministre': 0.0,
 'endomètre': 0.0,
 'asthme sévère': 0.0,
 'évaluation keytruda pembrolizumab merck traitement': 0.0,
 'utérus': 0.0,
 'aide à la décision': 0.0,
 'ensemble des critères': 0.0,
 'ensemble des critères prévus': 0.0,
 'usage d_alcool': 0.0,
 'excellence en santé': 0.0,
 'excellence en santé et en services sociaux': 0.0,
 'excellence en santé et en se

In [118]:
len(zeros) / len(vocabulaire) *100

6.970954356846473

In [119]:
terms_tfidf

{'traitement': 0.23369636532326207,
 'jamp': 0.29924686605391876,
 'évaluation': 0.20741661056128932,
 'médicaments': 0.3981305740253018,
 'demande': 0.5580812780469357,
 'inesss': 0.5529704133634528,
 'attente': 0.34497126734932826,
 'injection': 0.2569514183782643,
 'ministre': 0.40365828329161973,
 'patients': 0.21635323821345812,
 'médicament': 0.29438295880067566,
 'cours': 0.14865292382697187,
 'inscription': 0.15309853058123454,
 'sodium': 0.28981644539900886,
 'citrate': 0.08590160639011464,
 'attente d_évaluation': 0.2964632267509891,
 'cancer': 0.5095657182967378,
 'angita': 0.03674036534465795,
 'darunavir': 0.2871619287714037,
 'calcium': 0.5756092052559373,
 'lurasidone': 0.17342763328232366,
 'bicarbonate': 0.17363255382174503,
 'réponses': 0.4692707467706064,
 'dextrose': 0.17363255382174503,
 'sulfonate': 0.08681627691087251,
 'bicarbonate de sodium': 0.17363255382174503,
 'glycopyrrolate': 0.17363255382174503,
 'cours d_évaluation': 0.2594076785489053,
 'réponses rapid

In [120]:
terms_weighted = DataFrame(terms_tfidf.items(), columns=['Terme formatté', 'TF-IDF'])

In [121]:
terms_weighted.sort_values(["TF-IDF"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

terms_weighted = terms_weighted.drop_duplicates(keep='first')
terms_weighted

,Terme formatté,TF-IDF
5854,venofer,0.857335
4835,préservateur,0.844618
4931,vag,0.836657
835,bid,0.832799
5052,fulvestrant,0.832582
...,...,...
4904,évaluation diagnostique des troubles,0.000000
5636,hypertension pulmonaire,0.000000
2878,évaluation gavreto,0.000000
4910,évaluation en traumatologie,0.000000


In [122]:
terms_weighted = pd.merge(csv, terms_weighted, on='Terme formatté').drop_duplicates(
  subset = ['Terme', 'Fréquence (TF)'],
  keep = 'first').reset_index(drop = True)

terms_weighted

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm,Terme formatté,TF (sklearn),DF (sklearn),TF-IDF
0,traitement,NOM,7872,-,-,1218,False,True,traitement,7864,1180,0.233696
1,jamp,NOM,7296,-,-,992,False,False,jamp,2556,992,0.299247
2,évaluation,NOM,6785,-,-,1396,False,False,évaluation,1694,324,0.207417
3,médicaments,NOM,5448,-,-,1206,False,True,médicaments,5451,1206,0.398131
4,demande,NOM,3662,-,-,280,False,False,demande,3662,240,0.558081
...,...,...,...,...,...,...,...,...,...,...,...,...
6020,degré,NOM,16,-,-,14,False,False,degré,16,14,0.143549
6021,foam,NOM,16,-,-,8,False,False,foam,16,8,0.486611
6022,coque,NOM,16,-,-,22,False,True,coque,16,12,0.448285
6023,médicaments du québec,NOM PRP:det NOM,16,88.74281960037526,4.496265284875781e-21,13,False,False,médicaments du québec,16,13,0.119791


## **OKapi BM25**
https://hal.archives-ouvertes.fr/hal-00760158 

In [123]:
from rank_bm25 import BM25Okapi

In [124]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer("(\w+\'|\w+-\w+|\(|\)|\w+)")

bm25 = BM25Okapi([tokenizer.tokenize(t) for t in corpus])

In [125]:
vocabulaire = csv["Terme"].dropna().tolist()

vocabulaire

['traitement',
 'jamp',
 'évaluation',
 'médicaments',
 'demande',
 'inesss',
 'attente',
 'injection',
 'ministre',
 'patients',
 'médicament',
 'cours',
 'inscription',
 'sodium',
 'citrate',
 "attente d'évaluation",
 'cancer',
 'angita',
 'darunavir',
 'calcium',
 'lurasidone',
 'bicarbonate',
 'réponses',
 'dextrose',
 'sulfonate',
 'bicarbonate de sodium',
 'glycopyrrolate',
 "cours d'évaluation",
 'réponses rapides',
 'covid-19',
 'santé',
 'indication',
 'réponses rapides disponibles',
 'pré-ac',
 'services',
 'listes',
 'décision',
 'québec',
 'nom',
 'demande en attente',
 'coll',
 'recommandations',
 'adultes',
 'projet',
 'fabricant',
 'cliniques',
 'chlorhydrate',
 'soins',
 'forme',
 'dépôt',
 'pré-ac en attente',
 'conseil',
 'recommandation',
 'fumarate',
 'demande en cours',
 "demande en cours d'évaluation",
 'commune',
 'dénomination commune',
 'exception',
 'dénomination',
 'évaluations',
 'maladie',
 'listes des médicaments',
 'canada',
 'usage',
 'ans',
 'nom du fab

In [126]:
#tokenizer = RegexpTokenizer(r"\w\'|\w+")
## Revoir ça
tokenized_queries = [tokenizer.tokenize(str(t)) for t in vocabulaire]

features_names = [t for t in set(vocabulaire)]
corpus_index = [corpus.index(n) for n in corpus]

tab = [bm25.get_scores(query) for query in tokenized_queries]
df = pd.DataFrame(tab, index=features_names, columns=corpus_index).transpose()

In [127]:
print(len(tokenized_queries))

tokenized_queries

6025


[['traitement'],
 ['jamp'],
 ['évaluation'],
 ['médicaments'],
 ['demande'],
 ['inesss'],
 ['attente'],
 ['injection'],
 ['ministre'],
 ['patients'],
 ['médicament'],
 ['cours'],
 ['inscription'],
 ['sodium'],
 ['citrate'],
 ['attente', "d'", 'évaluation'],
 ['cancer'],
 ['angita'],
 ['darunavir'],
 ['calcium'],
 ['lurasidone'],
 ['bicarbonate'],
 ['réponses'],
 ['dextrose'],
 ['sulfonate'],
 ['bicarbonate', 'de', 'sodium'],
 ['glycopyrrolate'],
 ['cours', "d'", 'évaluation'],
 ['réponses', 'rapides'],
 ['covid-19'],
 ['santé'],
 ['indication'],
 ['réponses', 'rapides', 'disponibles'],
 ['pré-ac'],
 ['services'],
 ['listes'],
 ['décision'],
 ['québec'],
 ['nom'],
 ['demande', 'en', 'attente'],
 ['coll'],
 ['recommandations'],
 ['adultes'],
 ['projet'],
 ['fabricant'],
 ['cliniques'],
 ['chlorhydrate'],
 ['soins'],
 ['forme'],
 ['dépôt'],
 ['pré-ac', 'en', 'attente'],
 ['conseil'],
 ['recommandation'],
 ['fumarate'],
 ['demande', 'en', 'cours'],
 ['demande', 'en', 'cours', "d'", 'évalua

In [128]:
df

,retards liés,littérature scientifique,alk réévaluation,dav,mutations activatrices d'insertion,néphropathie chronique et de diabète,warfarine,stress,paramètres intermédiaires,registre des traumatismes,...,patients adultes atteints d'un cancer de l'endomètre,présoumission avec les fabricants préavis obligatoire,sécheresse vaginale,vraylar cariprazine abbvie traitement,cellules claires,nicolay ferrari,transition,folotyn,kio évaluations,summary
0,0.000000,0.0,1.413841,0.000000,1.709287,0.000000,0.0,0.0,2.267082,0.000000,...,3.071228,0.0,3.071228,0.0,0.0,0.0,0.0,0.0,3.308659,0.0
1,0.000000,0.0,3.192352,0.000000,3.132485,0.000000,0.0,0.0,2.568860,0.000000,...,3.160309,0.0,3.160309,0.0,0.0,0.0,0.0,0.0,3.173297,0.0
2,1.771574,0.0,2.391556,1.771574,2.384877,2.707759,0.0,0.0,1.771574,0.694350,...,4.241023,0.0,4.241023,0.0,0.0,0.0,0.0,0.0,5.024616,0.0
3,0.000000,0.0,0.000000,0.000000,2.332044,2.647774,0.0,0.0,1.732329,0.000000,...,3.204549,0.0,3.204549,0.0,0.0,0.0,0.0,0.0,3.261402,0.0
4,0.000000,0.0,0.000000,0.000000,0.000000,5.136411,0.0,0.0,3.580572,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030,2.060277,0.0,2.294128,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1.231825,...,4.169879,0.0,4.169879,0.0,0.0,0.0,0.0,0.0,2.938054,0.0
1466,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,3.101687,0.0,3.101687,0.0,0.0,0.0,0.0,0.0,2.729494,0.0
1468,0.000000,0.0,2.313594,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1.242277,...,4.192129,0.0,4.192129,0.0,0.0,0.0,0.0,0.0,0.678755,0.0
1520,0.000000,0.0,0.000000,0.000000,0.000000,2.889684,0.0,0.0,0.000000,0.000000,...,3.001874,0.0,3.001874,0.0,0.0,0.0,0.0,0.0,3.741918,0.0


In [129]:
#df.to_csv(base_path + titre + '_matrice-OkapiBM25.csv') # Si on veut avoir la matrice (mais le fichier peut être très volumineux)

In [130]:
terms_okapi = {term: df[term].max() for term in df}

In [131]:
terms_okapi

{'retards liés': 3.444671729341301,
 'littérature scientifique': 0.10701347960806883,
 'alk réévaluation': 3.591974032304352,
 'dav': 3.499849489030184,
 "mutations activatrices d'insertion": 4.7148847177222155,
 'néphropathie chronique et de diabète': 5.257841174613827,
 'warfarine': 6.132125678171774,
 'stress': 0.08295426080042402,
 'paramètres intermédiaires': 3.619849830471083,
 'registre des traumatismes': 1.8630766781171442,
 'brolucizumab novartis traitement': 3.402184291885258,
 'agoniste': 2.7748876529897464,
 'liquides': 4.473505741602462,
 'injections de vitamine': 0.1086563179178894,
 'erreurs innées': 0.08354895067069236,
 'hypertension artérielle hypolipémiants infections urinaires': 9.14919009307234,
 'kératose': 3.723853997531944,
 'cétirizine (chlorhydrate de)': 3.6279987222684102,
 'sclérose latérale amyotrophique': 0.17999355012521587,
 'juge': 0.15565505521368278,
 'degré': 0.15439718943768854,
 'hemlibra': 0.16184688085957297,
 'problèmes techniques': 5.0822985857

In [132]:
tab = DataFrame(terms_okapi.items(), columns=['Terme', 'OkapiBM25'])
tab

,Terme,OkapiBM25
0,retards liés,3.444672
1,littérature scientifique,0.107013
2,alk réévaluation,3.591974
3,dav,3.499849
4,mutations activatrices d'insertion,4.714885
...,...,...
6020,nicolay ferrari,5.809825
6021,transition,10.636027
6022,folotyn,9.754495
6023,kio évaluations,7.470046


In [133]:
tab.sort_values(["OkapiBM25"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

tab

,Terme,OkapiBM25
5003,itinérance,47.329810
5048,hormone de croissance pré-ac en cours,47.082489
5046,opdivo,43.677060
5612,activité,42.626487
5002,embryons,41.894857
...,...,...
4470,actikerall,0.000000
5242,influenza chez l'enfant et l'adulte,0.000000
1776,pembrolizumab,0.000000
33,attente d'évaluation oxlumo,0.000000


In [134]:
terms_weighted

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm,Terme formatté,TF (sklearn),DF (sklearn),TF-IDF
0,traitement,NOM,7872,-,-,1218,False,True,traitement,7864,1180,0.233696
1,jamp,NOM,7296,-,-,992,False,False,jamp,2556,992,0.299247
2,évaluation,NOM,6785,-,-,1396,False,False,évaluation,1694,324,0.207417
3,médicaments,NOM,5448,-,-,1206,False,True,médicaments,5451,1206,0.398131
4,demande,NOM,3662,-,-,280,False,False,demande,3662,240,0.558081
...,...,...,...,...,...,...,...,...,...,...,...,...
6020,degré,NOM,16,-,-,14,False,False,degré,16,14,0.143549
6021,foam,NOM,16,-,-,8,False,False,foam,16,8,0.486611
6022,coque,NOM,16,-,-,22,False,True,coque,16,12,0.448285
6023,médicaments du québec,NOM PRP:det NOM,16,88.74281960037526,4.496265284875781e-21,13,False,False,médicaments du québec,16,13,0.119791


In [135]:
tab = pd.merge(terms_weighted, tab, on="Terme")
tab.sort_values(["OkapiBM25"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

In [136]:
base_path = '../05-transformation/'

In [137]:
if tag:
    file_path = base_path + acteur + '_' + tag + '_weighting_OKapiBM25.csv'

else: 
    file_path = base_path + acteur  + '_weighting_OKapiBM25.csv'
tab.to_csv(file_path)

In [138]:
tab

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm,Terme formatté,TF (sklearn),DF (sklearn),TF-IDF,OkapiBM25
4287,itinérance,NOM,30,-,-,15,False,True,itinérance,12,8,0.472798,47.329810
2536,hormone de croissance pré-ac en cours,NOM PRP NOM NOM PRP NOM,44,534.9832950178793,2.3271021680629744e-118,44,False,False,hormone de croissance pré_ac en cours,44,44,0.006176,47.082489
379,opdivo,NOM,226,-,-,88,False,False,opdivo,214,86,0.429815,43.677060
1520,activité,NOM,71,-,-,374,False,False,activité,28,28,0.094159,42.626487
5994,embryons,NOM,16,-,-,7,False,False,embryons,16,7,0.505658,41.894857
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5561,jim boulanger direction scientifique elisabeth...,NOM ADJ NOM ADJ NOM VER:pper,18,382.1569619377114,4.2204638905297015e-85,18,False,False,jim boulanger direction scientifique elisabeth...,18,18,0.088114,0.000000
5237,pansements de base,NOM PRP NOM,20,321.0270241228294,8.653548940266297e-72,4,False,False,pansements de base,20,4,0.382036,0.000000
4992,actikerall,NOM,22,-,-,10,False,False,actikerall,22,10,0.581519,0.000000
890,sédation palliative continue,NOM ADJ ADJ,106,-inf,1.0,88,False,False,sédation palliative continue,106,88,0.285686,0.000000
